In [1]:
import pandas as pd 
from BinaryDecisionTreeConstraints.BinaryDecisionTreeConstraints import Node
import numpy as np

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [7]:
# Defining the X and Y matrices
Y = df_train['Survived'].values
X = df_train[['Sex','Cabin','Age','Fare','Pclass','Embarked']]
# Saving the feature list 
features = list(X.columns)
nb_stop = np.ceil(len(df_train) * 0.03)
list_constraints = [ ["0root",'Embarked',"categorical"]]
dataframe_constraints = pd.DataFrame(list_constraints)
dataframe_constraints = dataframe_constraints.rename(columns={0:"path",1:"feature",2:"type_feature"})
hp = {
    "max_depth": 4,
    "stop" : nb_stop,
    "dataframe_constraints": dataframe_constraints
}

root = Node(Y, X, **hp)

In [8]:
dataframe_constraints

,path,feature,type_feature
0,0root,Embarked,categorical


In [9]:
root.grow_tree()

C:\Users\kaci-\Anaconda3\envs\mynewenv\lib\site-packages\pandas\core\algorithms.py:465: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return f(comps, values)
C:\Users\kaci-\Anaconda3\envs\mynewenv\lib\site-packages\pandas\core\algorithms.py:465: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return f(comps, values)
C:\Users\kaci-\Anaconda3\envs\mynewenv\lib\site-packages\pandas\core\algorithms.py:465: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return f(comps, values)
C:\Users\kaci-\Anaconda3\envs\mynewenv\lib\site-packages\pandas\core\algorithms.py:465: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return f(comps, values)


0 Embarked ['S' 'Q'] categorical
1 Cabin ['A14' 'C128' 'C30' 'C46' 'C87' 'C91' 'C95' 'D26' 'D30' 'D46' 'C124' 'D50'
 'E31' 'E38' 'E46' 'E58' 'E63' 'E77' 'F G63' 'F G73' 'F38' 'D6' 'C110' 'T'
 'A32' 'B102' 'A36' 'B51 B53 B55' 'B38' 'B71' 'A24' 'B19' 'A19' 'B94'
 'C22 C26' 'C78' 'B22' 'C2' 'C23 C25 C27' 'E44' 'C123' 'E67' 'G6' 'C83'] categorical
2 Pclass 1.5 numerical


C:\Users\kaci-\Anaconda3\envs\mynewenv\lib\site-packages\pandas\core\algorithms.py:465: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return f(comps, values)
C:\Users\kaci-\Anaconda3\envs\mynewenv\lib\site-packages\pandas\core\algorithms.py:465: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return f(comps, values)
C:\Users\kaci-\Anaconda3\envs\mynewenv\lib\site-packages\pandas\core\algorithms.py:465: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return f(comps, values)
C:\Users\kaci-\Anaconda3\envs\mynewenv\lib\site-packages\pandas\core\algorithms.py:465: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return f(comps, values)
C:\Users\kaci-\Anaconda3\envs\mynewenv\l

3 Sex ['male'] categorical


C:\Users\kaci-\Anaconda3\envs\mynewenv\lib\site-packages\pandas\core\algorithms.py:465: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return f(comps, values)


In [10]:
root.print_tree_dataframe()

,path,rule,distribution,probabilite,prediction
0,0root,,"{0: 549, 1: 342}",0.384,0
1,0root1left_node,Embarked in ['S' 'Q'],"{0: 474, 1: 247}",0.343,0
2,0root1left_node2left_node,Cabin in ['A14' 'C128' 'C30' 'C46' 'C87' 'C91...,"{1: 12, 0: 50}",0.194,0
3,0root1left_node2right_node,Cabin not in ['A14' 'C128' 'C30' 'C46' 'C87' ...,"{0: 424, 1: 235}",0.357,0
4,0root1left_node2right_node3left_node,Pclass <= 1.5,"{1: 65, 0: 11}",0.855,1
5,0root1left_node2right_node3right_node,Pclass> 1.5,"{0: 413, 1: 170}",0.292,0
6,0root1left_node2right_node3right_node4left_node,Sex in ['male'],"{0: 346, 1: 52}",0.131,0
7,0root1left_node2right_node3right_node4right_node,Sex not in ['male'],"{1: 118, 0: 67}",0.638,1
8,0root1right_node,Embarked not in ['S' 'Q'],"{1: 95, 0: 75}",0.559,1


In [11]:
predictions = root.predict(df_test[['Sex','Cabin','Age','Fare','Pclass','Embarked']])

In [12]:
predictions

,prediction,probability
0,0,0.131
1,1,0.638
2,0,0.131
3,0,0.131
4,1,0.638
...,...,...
413,0,0.131
414,1,0.559
415,0,0.131
416,0,0.131


### rajouter la contrainte de valeur pour me dataframe constraints ( en gros faire la contrainte de la variable et de quoi elle serait égale ) + rajouter la condition du gain + trouver un moyen de plot le graph